In [1]:
import numpy as np
import os

log_dir = 'pi_log'
log_list = []
time_list = []
for _, _, files in os.walk(log_dir):
    for f in files:
        fname = os.path.join(log_dir, f)
        if fname.endswith('.log'):
            log_list.append(fname)
        else:
            time_list.append(fname)
print(time_list)
print(log_list)

['pi_log/Boudia_20210707T013618.time', 'pi_log/Ni_20210707T013943.time', 'pi_log/Homo_20210707T013853.time', 'pi_log/CircleGroup_20210707T013732.time']
['pi_log/log_2021-07-07T013943.log', 'pi_log/log_2021-07-07T013853.log', 'pi_log/log_2021-07-07T013732.log', 'pi_log/log_2021-07-07T013618.log']


In [2]:
import re

def parse_time(l):
    pattern = '[0-9]{4}-[0-9]{2}-[0-9]{2}T[0-9]{6}'
    base = os.path.basename(l)
    return re.search(pattern, base).group()

print(log_list[0])
t = parse_time(log_list[0])

print(t)

pi_log/log_2021-07-07T013943.log
2021-07-07T013943


In [3]:
def parse_block(block):
    m = {}
    m['func'] = block[0]
    meter_num = 0
    meter_time = 0
    control =['server', 'supplier', 'center']
    ta = ['ttp', 'kgc', 'ta']
    m['control'] = 0
    m['ta'] = 0
    m['agg'] = 0
    for r in block[2::2]:
        x, y = r.split(':')
        t = sum(int(e) for e in y.split(','))
        if x.startswith('meter_'):
            meter_num += 1
            meter_time += t
        elif x in control:
            m['control'] = +t
        elif x in ta:
            m['ta'] = t
        else:
            m[x] = t
    
    m['meter_num'] = meter_num
    m['meter_time'] = meter_time
    return m

def parse_record(lines):
    pattern = '[^ ]*$'
    host_pattern = '[^ ]*'
    name_pattern = 'bjut\.(.*)\.myMain'
    host = re.search(host_pattern, lines[0]).group()
    name = re.findall(name_pattern, lines[0])[0]

    func_block = []
    block = []
    for line in lines:
        s = re.search(pattern, line).group().strip()
        # print(s)
        # parse function block
        if ':' not in s: # enter function
            if len(block) != 0:
                func_block.append(block)
            block = []
            block.append(s)
        else:
            block.append(s)
    if len(block) != 0:
        func_block.append(block)

    # print(func_block[0])
    record_list = []
    for b in func_block:
        r = parse_block(b)
        # print(r)
        r['host'] = host
        r['name'] = name
        record_list.append(r)
    return record_list

In [4]:


import csv

fields_name = []
with open('data.csv', 'w', newline='') as csvfile:
    writer = None
    for l in log_list:
        log_time = parse_time(l)
        with open(l) as fo:
            record = parse_record(fo.readlines())
            if len(fields_name) == 0:
                fields_name  = fields_name + [k for k in record[0].keys()]
                fields_name.append('timestamp')
                writer = csv.DictWriter(csvfile, fieldnames=fields_name)
                writer.writeheader()
                print(fields_name)
            for l in record:
                l['timestamp'] = log_time
                writer.writerow(l)

{'func': 'oneTimeMeterRegTime', 'control': 22630346000, 'ta': 1136414, 'agg': 0, 'meter_num': 20, 'meter_time': 377390632}
{'func': 'oneTimeMeterRegTime', 'control': 33695378435, 'ta': 167855, 'agg': 0, 'meter_num': 30, 'meter_time': 541295615}
{'func': 'oneTimeMeterRegTime', 'control': 44772005255, 'ta': 264220, 'agg': 0, 'meter_num': 40, 'meter_time': 722366437}
{'func': 'oneTimeMeterRegTime', 'control': 55794820761, 'ta': 451681, 'agg': 0, 'meter_num': 50, 'meter_time': 901926590}
{'func': 'oneTimeMeterRegTime', 'control': 67722555052, 'ta': 591197, 'agg': 0, 'meter_num': 60, 'meter_time': 1092156080}
{'func': 'oneTimeMeterRegTime', 'control': 22507135869, 'ta': 191667, 'agg': 0, 'meter_num': 20, 'meter_time': 363257274}
{'func': 'oneTimeMeterRepTime', 'control': 71604551, 'ta': 0, 'agg': 4005201712, 'meter_num': 20, 'meter_time': 2208446468}
{'func': 'oneTimeMeterRegTime', 'control': 33991424412, 'ta': 305885, 'agg': 0, 'meter_num': 30, 'meter_time': 547091447}
{'func': 'oneTimeMet